In [350]:
# !pip install simpy

In [361]:
# GLOBAL VARIABLE DEFINES THE NUMBER OF MACHINES AVAILABLE AND SYSTEM TIME
SHIFT_TIME = 480

ARRIVAL_MIN, ARRIVAL_MAX = 8, 12

CLEAN_MIN, CLEAN_MAX = 15, 20
PRIMER_MIN, PRIMER_MAX = 25, 35
PAINT_MIN, PAINT_MAX = 30, 40

CLEAN_MACHINES = 1
PRIMER_MACHINES = 2
PAINT_MACHINES = 1


In [352]:
# Imported necessary libraries
import simpy
import random
import statistics


In [353]:
car_count = 0                 # Total cars that entered the system
completed_cars = 0            # Cars that completed all stations

# System performance metrics
system_times = []             # Time spent by each car in the system
alerts_triggered = 0          # Number of bottleneck alerts raised

# Queue statistics
queue_max = {
    "Cleaning": 0,
    "Primer": 0,
    "Painting": 0
}

# Waiting time at each station
wait_times = {
    "Cleaning": [],
    "Primer": [],
    "Painting": []
}

# Busy time (used for utilization calculation)
busy_time = {
    "Cleaning": 0,
    "Primer": 0,
    "Painting": 0
}


In [354]:
# SimPy resources for each station
def create_stations(env):

    # Capacity = number of machines available.

    return (
        simpy.Resource(env, capacity=CLEAN_MACHINES),
        simpy.Resource(env, capacity=PRIMER_MACHINES),
        simpy.Resource(env, capacity=PAINT_MACHINES),
    )


In [355]:
def process_station(env, car_name, station_name, resource, min_t, max_t,
                    cleaning, primer, painting):
    """
    Simulates a single car passing through one station.
    Handles queueing, processing, metric collection, and real-time logging.
    """
    global alerts_triggered

    # Log the moment the car arrives at this station along with current queue status
    print(
        f"{car_name} arrived at {station_name} at time {env.now:.1f} | "
        f"{q_status(cleaning, primer, painting)}"
    )

    with resource.request() as req:

        # Capture queue length before acquiring the machine
        q_len = len(resource.queue)
        queue_max[station_name] = max(queue_max[station_name], q_len)

        # Trigger alert if queue length crosses threshold
        if q_len > 3:
            alerts_triggered += 1
            print(
                f"ALERT: Queue at {station_name} has "
                f"{q_len} cars waiting at time {env.now:.1f}"
            )

        # Measure waiting time until the car gets the machine
        wait_start = env.now
        yield req
        wait_times[station_name].append(env.now - wait_start)

        # Log start of processing with live queue status
        print(
            f"{car_name} started {station_name} at {env.now:.1f} | "
            f"{q_status(cleaning, primer, painting)}"
        )

        # Sample processing time using uniform distribution
        process_time = random.uniform(min_t, max_t)

        # Track machine busy time only within shift duration
        start = env.now
        end = start + process_time
        if start < SHIFT_TIME:
            busy_time[station_name] += min(end, SHIFT_TIME) - start

        # Simulate processing delay
        yield env.timeout(process_time)

        # Log completion of processing at this station
        print(
            f"{car_name} finished {station_name} at {env.now:.1f} | "
            f"{q_status(cleaning, primer, painting)}"
        )


In [356]:
def car(env, name, cleaning, primer, painting):

    global completed_cars

    # Record the time when the car enters the system
    arrival = env.now
    print(
        f"{name} arrived at time {env.now:.1f} | "
        f"{q_status(cleaning, primer, painting)}"
    )

    # Pass through Cleaning station
    yield from process_station(
        env, name, "Cleaning", cleaning, CLEAN_MIN, CLEAN_MAX,
        cleaning, primer, painting
    )

    # Pass through Primer station
    yield from process_station(
        env, name, "Primer", primer, PRIMER_MIN, PRIMER_MAX,
        cleaning, primer, painting
    )

    # Pass through Painting station
    yield from process_station(
        env, name, "Painting", painting, PAINT_MIN, PAINT_MAX,
        cleaning, primer, painting
    )

    # Update completion metrics once the car exits all stations
    completed_cars += 1
    system_times.append(env.now - arrival)

    # Log the car’s exit from the system with final queue snapshot
    print(
        f"{name} exited system at {env.now:.1f} | "
        f"{q_status(cleaning, primer, painting)}"
    )


In [357]:
def car_generator(env, cleaning, primer, painting):

    global car_count

    while True:
        inter = random.uniform(ARRIVAL_MIN, ARRIVAL_MAX)

        # No new cars are added after SHIFT_TIME, but existing cars finish processing
        if env.now + inter > SHIFT_TIME:
            break

        yield env.timeout(inter)

        car_count += 1
        env.process(car(env, f"Car-{car_count}", cleaning, primer, painting))


In [358]:
def q_status(cleaning, primer, painting):
    return (
        f"Q(Cleaning={len(cleaning.queue)}, "
        f"Primer={len(primer.queue)}, "
        f"Painting={len(painting.queue)})"
    )


In [359]:
env = simpy.Environment()

# Initialize station resources
cleaning, primer, painting = create_stations(env)

# Start the car arrival process
env.process(car_generator(env, cleaning, primer, painting))

# Run the simulation until all cars complete
env.run()


Car-1 arrived at time 8.6 | Q(Cleaning=0, Primer=0, Painting=0)
Car-1 arrived at Cleaning at time 8.6 | Q(Cleaning=0, Primer=0, Painting=0)
Car-1 started Cleaning at 8.6 | Q(Cleaning=0, Primer=0, Painting=0)
Car-2 arrived at time 17.5 | Q(Cleaning=0, Primer=0, Painting=0)
Car-2 arrived at Cleaning at time 17.5 | Q(Cleaning=0, Primer=0, Painting=0)
Car-1 finished Cleaning at 25.3 | Q(Cleaning=1, Primer=0, Painting=0)
Car-1 arrived at Primer at time 25.3 | Q(Cleaning=1, Primer=0, Painting=0)
Car-1 started Primer at 25.3 | Q(Cleaning=0, Primer=0, Painting=0)
Car-2 started Cleaning at 25.3 | Q(Cleaning=0, Primer=0, Painting=0)
Car-3 arrived at time 27.5 | Q(Cleaning=0, Primer=0, Painting=0)
Car-3 arrived at Cleaning at time 27.5 | Q(Cleaning=0, Primer=0, Painting=0)
Car-4 arrived at time 36.7 | Q(Cleaning=1, Primer=0, Painting=0)
Car-4 arrived at Cleaning at time 36.7 | Q(Cleaning=1, Primer=0, Painting=0)
Car-2 finished Cleaning at 40.4 | Q(Cleaning=2, Primer=0, Painting=0)
Car-2 arrived a

In [360]:
print("\n===== Paint Shop Simulation Results =====")

# env.now gives the actual time when the last car exits the system
print(f"Simulation ended at: {env.now:.2f} minutes")
print(f"Total cars completed: {completed_cars}")

# Average total time spent by a car in the system
print(f"Average system time per car: {statistics.mean(system_times):.2f} minutes\n")

# Station capacities (needed for utilization calculation)
capacity = {
    "Cleaning": CLEAN_MACHINES,
    "Primer": PRIMER_MACHINES,
    "Painting": PAINT_MACHINES
}

for station in ["Cleaning", "Primer", "Painting"]:

    # Utilization = busy time during shift / total available machine time
    utilization = (busy_time[station] / (SHIFT_TIME * capacity[station])) * 100

    avg_wait = statistics.mean(wait_times[station]) if wait_times[station] else 0

    print(f"{station} Station:")
    print(f"- Utilization: {utilization:.2f}%")
    print(f"- Max queue length: {queue_max[station]}")
    print(f"- Avg waiting time: {avg_wait:.2f} minutes\n")

print(f"Alerts triggered: {alerts_triggered}")



===== Paint Shop Simulation Results =====
Simulation ended at: 1698.42 minutes
Total cars completed: 47
Average system time per car: 645.15 minutes

Cleaning Station:
- Utilization: 98.21%
- Max queue length: 19
- Avg waiting time: 163.48 minutes

Primer Station:
- Utilization: 84.18%
- Max queue length: 1
- Avg waiting time: 0.25 minutes

Painting Station:
- Utilization: 89.16%
- Max queue length: 23
- Avg waiting time: 398.22 minutes

Alerts triggered: 79
